In [1]:
import azureml.core
print(azureml.core.VERSION)

1.0.23


Creating the workspace

In [ ]:
from azureml.core import Workspace
ws = Workspace.create(name='MovieSuccessPrediction',
                      subscription_id='daf480ce-c31b-4cf4-8701-1ecca3e5ab9f',
                      resource_group='docs',
                      create_resource_group=True,
                      location='eastus' 
                     )

In [7]:
ws.get_details()

{'id': '/subscriptions/daf480ce-c31b-4cf4-8701-1ecca3e5ab9f/resourceGroups/docs/providers/Microsoft.MachineLearningServices/workspaces/myworkspace',
 'name': 'myworkspace',
 'location': 'eastus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'workspaceid': '87425cf8-ee66-43b5-8c2d-fc139102ee3b',
 'description': '',
 'friendlyName': 'myworkspace',
 'creationTime': '2019-04-14T22:51:28.6650227+00:00',
 'containerRegistry': '/subscriptions/daf480ce-c31b-4cf4-8701-1ecca3e5ab9f/resourcegroups/docs/providers/microsoft.containerregistry/registries/myworkspacrmczfxxtn',
 'keyVault': '/subscriptions/daf480ce-c31b-4cf4-8701-1ecca3e5ab9f/resourcegroups/docs/providers/microsoft.keyvault/vaults/myworkspkeyvaultbizmtsfm',
 'applicationInsights': '/subscriptions/daf480ce-c31b-4cf4-8701-1ecca3e5ab9f/resourcegroups/docs/providers/microsoft.insights/components/myworkspinsightshuelhtnu',
 'identityPrincipalId': 'bb883a70-c58c-4223-b581-af8706b4584a',
 'identityTenantId': 'dbd6664d-4eb9-46eb-9

In [8]:
# Create the configuration file.
ws.write_config()

# Use this code to load the workspace from 
# other scripts and notebooks in this directory.
# ws = Workspace.from_config()

Register a trained model

In [9]:
from azureml.core.model import Model

model = Model.register(model_path = "model.pkl",
                       model_name = "model",
                       tags = {"key": "0.1"},
                       description = "test",
                       workspace = ws)

Registering model model


In [19]:
#Creating environment file

from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [20]:
from azureml.core.image import ContainerImage

# Image configuration
image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                 runtime = "python",
                                                 conda_file = "myenv.yml"
                                                 )

In [21]:
image_config.base_image = "mcr.microsoft.com/azureml/o16n-sample-user-base/ubuntu-miniconda"

In [22]:
# Register the image from the image configuration
image = ContainerImage.create(name = "myimage",
                              models = [model], #this is the model object
                              image_config = image_config,
                              workspace = ws
                              )

Creating image


In [28]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags={"data": "movie_metadata",  "method" : "sklearn"},
                                               description = 'Movie Success Predicton')

Deploying the image to a web service

In [29]:
from azureml.core.webservice import Webservice
service = Webservice.deploy_from_model(workspace=ws,
                                       name='model',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)




service.wait_for_deployment(show_output = True)
print(service.state)

Creating image
Image creation operation finished for image model:1, operation "Succeeded"
Creating service
Running..................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy


In [30]:
print(service.scoring_uri)

http://52.151.245.245:80/score


In [36]:
#Quick Test

import json

test_sample = json.dumps({'data': [
    [1900,0,1728,0,237,0,82,0,33]
]})
test_sample = bytes(test_sample,encoding = 'utf8')

prediction = service.run(input_data = test_sample)
print(prediction)

[3.0]
